In [ ]:
import pandas as pd
import numpy as np
import holidays
import calendar

In [ ]:
csv = "combined_data_MP_NE_dT.csv"
df = pd.read_csv(csv)
df

## column selection, drop columns not needed

In [ ]:
#df = df.drop(['startClusterName', 'startClusterZip', 'endClusterName', 'endClusterZip', 'endClusterID', 'distance'], axis=1 )
#df

In [ ]:
df.dtypes

In [ ]:
df['startClusterID'] = df['startClusterID'].astype(str)

In [ ]:
#df = df.groupby(['year-month','startClusterID', 'isSchoolHoliday', 'weekday', 'daytime'])['count'].sum().reset_index()
#df

# Counting Occurrence of each Weekday per Month

In [ ]:
# Create a date range from January 1, 2019, to January 31, 2022
start_date = '2019-01-01'
end_date = '2022-01-31'
date_range = pd.date_range(start=start_date, end=end_date, freq='D')

# Create a DataFrame with 'date' and 'weekday' columns
df_all_dates = pd.DataFrame({'date': date_range, 'weekday': date_range.day_name()})
df_all_dates['date'] = df_all_dates['date'].astype(str)
df_all_dates['normal_day'] = 1
df_all_dates

# School Holidays

In [ ]:
#read excel file with school holidays
df_s_hol_dates = pd.read_excel('school_holiday_ni_2019_2022.xlsx')

# Combine the columns "year," "month," and "day" into one column "date"
df_s_hol_dates['date'] = df_s_hol_dates.apply(lambda row: f"{row['year']}-{row['month']:02d}-{row['day']:02d}", axis=1)

# Drop the other columns "year," "month," and "day"
df_s_hol_dates.drop(columns=['year', 'month', 'day'], inplace=True)

# Convert the "date" column to pandas datetime and extract the weekday
df_s_hol_dates['weekday'] = pd.to_datetime(df_s_hol_dates['date']).dt.day_name()

df_s_hol_dates['School_Holiday'] = 1

df_s_hol_dates

In [ ]:
# Merge the DataFrames based on 'date' and 'weekday' and perform a left join
df_all_s_dates = pd.merge(df_all_dates, df_s_hol_dates, on=['date', 'weekday'], how='left')
df_all_s_dates.fillna(0, inplace=True)
df_all_s_dates['normal_day'] -= df_all_s_dates['School_Holiday']
df_all_s_dates

# Public Holidays

In [ ]:
p_holidays = holidays.country_holidays('DE', subdiv='NI')
for year in [2019,2020,2021,2022]:
    str(year) in p_holidays

list_p_holidays = list(p_holidays.items())
list_p_holidays

In [ ]:
df_p_holidays = pd.DataFrame(columns=['date', 'weekday'])

for i in range (0, len(list_p_holidays)):
    df_p_holidays.loc[len(df_p_holidays.index)]= list_p_holidays[i][0]

df_p_holidays['weekday'] = df_p_holidays['weekday'].apply(lambda x: x.strftime('%A'))
df_p_holidays['Public_Holiday'] = 1
df_p_holidays = df_p_holidays[df_p_holidays['date'] <= pd.to_datetime("2022-01-31").date()]
df_p_holidays['date'] = df_p_holidays['date'].astype(str)
df_p_holidays

# Merge public with school holidays

In [ ]:
df_merged = pd.merge(df_all_s_dates, df_p_holidays, on=['date', 'weekday'], how='outer')
df_merged.fillna(0, inplace=True)
df_merged

In [ ]:
df_merged['S_P_Holiday'] = np.minimum(df_merged['School_Holiday'], df_merged['Public_Holiday'])
df_merged['School_Holiday'] -= df_merged['S_P_Holiday']
df_merged['Public_Holiday'] -= df_merged['S_P_Holiday']
df_merged

In [ ]:
df_merged['normal_day'] -= df_merged['Public_Holiday']
df_merged

In [ ]:
column_sums = df_merged.sum(axis=0)
column_sums

In [ ]:
# Convert the "date" column to a pandas datetime type
df_merged['date'] = pd.to_datetime(df_merged['date'])

# Group the DataFrame by month and weekday and calculate the sum for other columns
df_grouped_by_month_weekday = df_merged.groupby([df_merged['date'].dt.to_period('M'), 'weekday']).agg({'normal_day': 'sum', 'School_Holiday': 'sum', 'Public_Holiday': 'sum', 'S_P_Holiday': 'sum'})

# Reset the index
#df_grouped_by_month_weekday = df_grouped_by_month_weekday.reset_index()

df_grouped_by_month_weekday

In [ ]:
# Find the Sunday of each month and update the values for "Public_Holiday" and "S_P_Holiday" if it exists
for month, group in df_grouped_by_month_weekday.groupby(level=0):
    sunday_values = group.loc[month, ['Public_Holiday', 'S_P_Holiday']].sum()
    df_grouped_by_month_weekday.loc[(month, 'Sunday'), ['Public_Holiday', 'S_P_Holiday']] += sunday_values

df_grouped_by_month_weekday

In [ ]:
# Find the Sunday of each month and update the values for "normal_day" and "School_Holiday" if it exists
for month, group in df_grouped_by_month_weekday.groupby(level=0):
    sunday_values = group.loc[(month, 'Sunday'), ['Public_Holiday']].sum()
    df_grouped_by_month_weekday.loc[(month, 'Sunday'), ['normal_day']] += sunday_values

    sunday_values = group.loc[(month, 'Sunday'), ['S_P_Holiday']].sum()
    df_grouped_by_month_weekday.loc[(month, 'Sunday'), ['School_Holiday']] += sunday_values

df_grouped_by_month_weekday

In [ ]:
column_sums = df_grouped_by_month_weekday.sum(axis=0)
column_sums

In [ ]:
# Reset the index to remove the 'weekday' index level
df_grouped_by_month_weekday = df_grouped_by_month_weekday.reset_index()
df_grouped_by_month_weekday

In [ ]:
df_grouped_by_month_weekday.drop(columns=['Public_Holiday', 'S_P_Holiday'], inplace=True)
df_grouped_by_month_weekday

# combine with df

In [ ]:
df

In [ ]:
df['weekday'].unique()

In [ ]:
df_grouped_by_month_weekday['weekday'].unique()

In [ ]:
df_grouped_by_month_weekday.replace({'weekday':{'Sunday': 'Sunday/Holiday'}}, inplace=True)
df_grouped_by_month_weekday['weekday'].unique()

In [ ]:
df_grouped_by_month_weekday.rename(columns={'date': 'year-month'}, inplace=True)
df_grouped_by_month_weekday['year-month'] = df_grouped_by_month_weekday['year-month'].astype(str)
df_grouped_by_month_weekday

In [ ]:
# Convert month to strings with leading zeros
df['month'] = df['month'].apply(lambda x: str(x).zfill(2))

# Concatenate 'year' and 'month' to form 'year-month'
df['year-month'] = df['year'].astype(str) + '-' + df['month']
df

In [ ]:
df_merged = pd.merge(df, df_grouped_by_month_weekday, on=['year-month', 'weekday'], how='left')
df_merged

In [ ]:
contains_nan = df_merged.isnull().any()
contains_nan

In [ ]:
# Create a custom function to calculate "count_corrected" based on conditions
def calculate_count_corrected(row):
    if row['isSchoolHoliday']:
        divider = row['School_Holiday']
    else:
        divider = row['normal_day']

    if divider == 0:
        return row['count']
    else:
        return row['count'] / divider

# Apply the custom function to create the "count_corrected" column
df_merged['count_corrected'] = df_merged.apply(calculate_count_corrected, axis=1)
df_merged['count_corrected'] = df_merged['count_corrected'].round(2)
df_merged

In [ ]:
df_merged.dtypes

In [ ]:
df_merged.drop(columns=['School_Holiday', 'normal_day'], inplace=True)
df_merged

In [ ]:
df_merged['month'] = df_merged['month'].astype(int)
df_merged

In [ ]:
df_merged.drop(columns='year-month', inplace=True)

In [ ]:
df_merged.to_csv(csv.split(sep='.')[0] + '_cC' + '.csv', index=False)

In [ ]:
description = df_merged['count_corrected'].describe()
description = description.round(2)
description

In [ ]:
description = df_merged['count'].describe()
description = description.round(2)
description

In [ ]:
sid = '03254028001013'
eid = '03254028001003'
filtered_df = df_merged[
    (df_merged['startClusterID'] == sid) &
    (df_merged['endClusterID'] == eid) &
    (df_merged['year'] == 2019) &
    (df_merged['month'] == 6)
    ]
filtered_df

In [ ]:
filtered_df_grouped = filtered_df.groupby(['year', 'month', 'startClusterID', 'endClusterID', 'weekday', 'isSchoolHoliday', 'normal_day', 'School_Holiday']).agg({
    'count': 'sum',
    'count_corrected': 'sum'
})
filtered_df_grouped

In [ ]:
df_filtered = df[
    (df['startClusterID'] == '03254028001013') &
    (df['endClusterID'] == '03254028001003') &
    (df['year'] == 2019) &
    (df['month'] == 6)
    ]
df_filtered

In [ ]:
create Dataframe (DF) with all dates for time span
input file with school holiday dates
merge DF from l1 and l2
create DF with public holiday dates in time span
merge with DF from l3
create attributes to count different types of days (normal, schoolHoliday, publicHoliday, schoolPublicHoliday)
assign sPH to min(sH, pH) and subtract sPH from sH and pH
find weekdays != Sunday that are pH or sPH and add their count to Sunday of same month
drop pH and sPH
input CSV file and merge with l5
create new attribute count_corrected and assign it values by dividing count by normal or sH
drop normal and sH
voila


In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
#matplotlib.rc_file_defaults()
mpl.rcParams.update(mpl.rcParamsDefault)
# Set the font family to Arial
mpl.rcParams['font.family'] = 'sans-serif'
mpl.rcParams['font.sans-serif'] = ['Arial']

dpi = 500
font_title = 16
font_label = 15
font_chart = 13
weight_label = 'bold'
weight_title = 'bold'
pad_label = 10
alpha = 0.7

In [ ]:
# Create a figure and axes
plt.figure(figsize=(6, 4))

# Create a box plot for 'count' and 'count_corrected'
plt.boxplot([df_merged['count'], df_merged['count_corrected']], vert=False)

# Set y-axis labels
plt.yticks([1, 2], ['count', 'count_corrected'], fontsize=font_chart)
plt.ylabel('Attribute', fontsize=font_label, labelpad=pad_label, weight=weight_label)
plt.xticks(fontsize=font_chart)
plt.xlabel('Value', fontsize=font_label, labelpad=pad_label, weight=weight_label)

# Set plot title
plt.title('Box Plot for Count and Count Corrected',fontsize=font_title, weight=weight_title)
plt.savefig('/Users/timon/Documents/ba/abbildungen/' + 'comparison_count_countCorrected.png', dpi=dpi)

# Show the plot
plt.show()